# Notebook with Various Illustrations into Papers

In [ ]:
%pip install --upgrade --editable ..

%load_ext autoreload
%autoreload 2
import mglyph as mg

## Siplest Glyphs that were there in the beginning

In [ ]:
import math

def simple_line(x: float, canvas:mg.Canvas) -> None:
    canvas.line((mg.lerp(x, 0, -1), 0), (mg.lerp(x, 0, 1), 0),
                width='50p', color='navy', linecap='round')
mg.show(simple_line, x=[55], values=False, scale=10)


def simple_scaled_square(x: float, canvas: mg.Canvas) -> None:
  top = mg.lerp(x, canvas.ycenter, canvas.ytop)
  bottom = mg.lerp(x, canvas.ycenter, canvas.ybottom)
  left = mg.lerp(x, canvas.xcenter, canvas.xleft)
  right = mg.lerp(x, canvas.xcenter, canvas.xright)
  canvas.polygon([(left, top), (right, top), (right, bottom), (left, bottom)], 
                 width='25p', linejoin='round', color='darkslategray', style='stroke')
mg.show(simple_scaled_square, x=[55], values=False, scale=10)


def simple_scaled_circle(x: float, canvas: mg.Canvas) -> None:
    canvas.circle(canvas.center, mg.lerp(x, 0.01, 1), color='darkred', style='stroke', width='25p')
mg.show(simple_scaled_circle, x=[55], values=False, scale=10)


def simple_scaled_star(x: float, canvas: mg.Canvas) -> None:
    radius = mg.lerp(x, 0.2, canvas.ysize/2)
    vertices = []
    for segment in range(5):
        vertices.append(mg.orbit(canvas.center, segment * 2*math.pi/5, radius))
        vertices.append(mg.orbit(canvas.center, (segment + 0.5) * 2*math.pi/5, math.cos(2*math.pi/5)/math.cos(math.pi/5) * radius))
    canvas.polygon(vertices, linejoin='round', color='indigo', style='stroke', width='25p')
mg.show(simple_scaled_star, x=[55], values=False, scale=10)


## One-Handed Clock

In [ ]:
def one_handed_clock(x: float, canvas: mg.Canvas) -> None:
  canvas.circle(canvas.center, radius=canvas.ysize/30, color='darkslategray', style='stroke', width='20p')
  angle = -x/100 * 2*math.pi
  canvas.line(mg.orbit(canvas.center, angle+math.pi, 0.18 * canvas.ysize/2),
              mg.orbit(canvas.center, angle, 1.0*canvas.ysize/2),  
              width='10p', linecap='round', color='maroon')

def one_handed_clock_with_hticks(x: float, canvas: mg.Canvas) -> None:
  for hour in range(12):
    canvas.line(mg.orbit(canvas.center, hour*math.pi/6, 0.8 * canvas.ysize/2),
                mg.orbit(canvas.center, hour*math.pi/6, canvas.ysize/2),
                         linecap='round', color=(0,0,0.5), width='38p')
    canvas.line(mg.orbit(canvas.center, hour*math.pi/6, 0.8 * canvas.ysize/2),
                mg.orbit(canvas.center, hour*math.pi/6, canvas.ysize/2),
                         linecap='round', color='white', width='31p')
  one_handed_clock(x, canvas)

def one_handed_clock_with_mticks(x: float, canvas: mg.Canvas) -> None:
  for minute in range(60):
    canvas.line(mg.orbit(canvas.center, minute*math.pi/30, 0.88*canvas.ysize/2),
                mg.orbit(canvas.center, minute*math.pi/30, 1.0*canvas.ysize/2), 
                width='5p', linecap='round', color='black')
  one_handed_clock_with_hticks(x, canvas)

mg.show(one_handed_clock_with_hticks, scale=10,
        x=[5, 25, 50, 75, 95])
mg.show(one_handed_clock, scale=10, values=False,
        x=[20, 21, 22, 23, 24])
mg.show(one_handed_clock_with_mticks, scale=10,
        x=[20, 21, 22, 23, 24])

## Dots -- Illegal and Legal

In [ ]:
def hundred_dots(x: float, canvas: mg.Canvas) -> None:
    base = 10
    for row in range(base):
        for col in range(base):
            if (row*base + col) <= mg.lerp(x, 0, base**2):
                canvas.circle(((col/(base-1))*canvas.xsize+canvas.xleft, (row/(base-1))*canvas.ysize+canvas.ytop),
                    radius=canvas.ysize*0.021, color='midnightblue')
            else:
                canvas.circle(((col/(base-1))*canvas.xsize+canvas.xleft, (row/(base-1))*canvas.ysize+canvas.ytop),
                    radius=canvas.ysize*0.009, color='lightslategray')
mg.show(hundred_dots, scale=10, values=False, x=[5, 25, 50, 52, 54])


from scipy.stats import qmc
def halton_sequence(x: float, canvas: mg.Canvas) -> None:
    sampler = qmc.Halton(d=2, scramble=False)
    sampler.fast_forward(1)
    n = int(mg.lerp(x, 0, 1500))
    points = sampler.random(n=n)
    for i, point in zip(range(n), points):
        canvas.circle((point[0]*canvas.xsize+canvas.xleft, point[1]*canvas.ysize+canvas.ytop),
                      radius=canvas.ysize*0.013, color='midnightblue')
mg.show(halton_sequence, scale=10, x=[5, 25, 50, 52, 54])